# Problema Mochila Multipla

* O problema de múltiplas mochilas é um problema de otimização, ou seja, uma função que deve ser maximizada ou minimizada e restrições que devem ser satisfeitas. Neste caso, além de envolver diversos objetos, também aplicado o uso de múltiplas mochilas. 

* Utilizaremos a biblioteca MIP do Python, é uma coleção de ferramentas para  modelagem e solução de programas lineares inteiros mistos (Mixed-Integer Linear programs). Utilizaremos para definir um modelo de solução para o problema.

In [87]:
# Importando bibliotecas
!pip install mip

from mip import *
import random


# Funções do Algoritmo

* Um aspecto muito importante do algoritmo genético é o uso de probabilidade, que introz um elemento aleatório (randômico) para o comportamento do algoritmo. A seguir será definido um valor constante a função random. É valido lembrar, que para valores diferentes, o algoritmo também dará resultados diferentes.

In [88]:
# Gerar produtos com peso e valor aleatórios
def gerar_produtos(prod):
    random.seed(random_seed)
    for i in range(num_produtos):
        cod = 'produto_{}'.format(i)
        valor = random.choice(range(1,10)) # Valor em reais
        peso = random.choice(range(100,999)) # Valor em reais
        prod[cod] = {'valor': valor,
            'peso': peso}

In [89]:
# Gera mochilas com pesos aleatórias
def gerar_mochilas(moc):
    random.seed(random_seed)
    for i in range(num_mochilas):
        cod = 'mochila_{}'.format(i)
        carga_maxima = random.choice(range(500,2000)) # Carga máxima em gramas
        moc[cod] = {'carga_maxima': carga_maxima}

* Criaremos funções que auxiliam na impressão dos dados de maneira organizada, através de interações dentro dos arrays

In [90]:
# Imprimir os produtos
def imprimir_produtos(prod):
    print("PRODUTOS")
    print("Cód \tValor \t\tPeso")
    for p in prod:
        print("{}\tR$ {},00 \t{}g".format(p, prod[p]['valor'], prod[p]['peso']))

In [91]:
# Imprime as mochilas
def imprimir_mochilas(moc):
    print("\nMOCHILAS")
    print("Cód \tCarga Máxima")
    for m in moc:
        print("{} \t{}g".format(m, moc[m]['carga_maxima']))

# Implementação do algoritmo

In [92]:
# Constantes do algoritmo genético
random_seed = 1
num_produtos = 18
num_mochilas = 3

In [93]:
# Variáveis que armazena os Dados
produtos = {}
mochilas = {}

#Criar produtos e mochilas
gerar_produtos(produtos)
gerar_mochilas(mochilas)

In [94]:
# Todos os produtos
imprimir_produtos(produtos)


PRODUTOS
Cód 	Valor 		Peso
produto_0	R$ 3,00 	682g
produto_1	R$ 2,00 	361g
produto_2	R$ 2,00 	607g
produto_3	R$ 8,00 	583g
produto_4	R$ 7,00 	907g
produto_5	R$ 4,00 	196g
produto_6	R$ 8,00 	129g
produto_7	R$ 7,00 	543g
produto_8	R$ 1,00 	812g
produto_9	R$ 8,00 	372g
produto_10	R$ 4,00 	705g
produto_11	R$ 2,00 	425g
produto_12	R$ 1,00 	122g
produto_13	R$ 1,00 	765g
produto_14	R$ 9,00 	109g
produto_15	R$ 7,00 	802g
produto_16	R$ 4,00 	532g
produto_17	R$ 1,00 	640g


In [95]:
# Todas as mochilas
imprimir_mochilas(mochilas)


MOCHILAS
Cód 	Carga Máxima
mochila_0 	775g
mochila_1 	1665g
mochila_2 	629g


* Aplicando a função model da biblioteca MIP, utilizando o parametro sense para maximização, conforme objetivo do algoritmo.

In [96]:
# Modelo 
modelo = Model(sense=MAXIMIZE)

In [97]:
# Variáveis de decisão
carga = {} # qual produto será colcoado em cada mochila

for m in mochilas:
    for p in produtos:
        carga[(m, p)] = modelo.add_var(var_type=BINARY)  

* As restrições e as quantidades de produtos e mochilas é o que difere um problema do outro. Neste caso, as restrições são listas a seguir:

In [98]:
# Restrições
# a) O mesmo produto não pode ser colocado nas três mochilas
for p in produtos:        
    modelo += xsum(carga[(m, p)] for m in mochilas) <= 1

# b) A soma dos pesos dos produtos alocados em uma mochila não devem ser maior do que a carga máxima suportada pela mochila
for m in mochilas:    
    modelo += xsum(carga[(m, p)] * produtos[p]['peso'] for p in produtos) <= mochilas[m]['carga_maxima']

* Como dito no inicio do projeto, o objetivo do algoritmo da mochila multipla é maximizar a solução, logo aplicamos a função maximize() no modelo gerado para cada item. Após, é aplicado a função optimize que tem o objetivo de otimizar o modelo da melhor forma possível.  

In [99]:
# Função Objetivo
modelo.objective = maximize(
    xsum(carga[(m, p)] * produtos[p]['valor']
         for m in mochilas 
             for p in produtos
         if (m, p) in carga))

modelo.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [100]:
# Resultado
print("\n****")
print("Valor Total em Todas as Mochilas {}".format(modelo.objective_values))
print("****")
for m in mochilas:
    print("\nCarga da Mochila {} com capacidade de {}g".format(m, mochilas[m]['carga_maxima']))
    valor_total = 0
    carga_total = 0
    for p in produtos:
        if (carga[(m, p)].x == 1):
            valor_total += produtos[p]['valor']
            carga_total += produtos[p]['peso']
            print("{} \tR$ {},00 \t{}g".format(p, produtos[p]['valor'], produtos[p]['peso']))
    
    print("-\nValor Total: R$ {},00 \nCarga Total: {}g\nCapacidade Ociosa: {}g".format(valor_total, carga_total, (mochilas[m]['carga_maxima']-carga_total)))


****
Valor Total em Todas as Mochilas [-52.0]
****

Carga da Mochila mochila_0 com capacidade de 775g
produto_3 	R$ 8,00 	583g
produto_14 	R$ 9,00 	109g
-
Valor Total: R$ 17,00 
Carga Total: 692g
Capacidade Ociosa: 83g

Carga da Mochila mochila_1 com capacidade de 1665g
produto_5 	R$ 4,00 	196g
produto_6 	R$ 8,00 	129g
produto_9 	R$ 8,00 	372g
produto_12 	R$ 1,00 	122g
produto_15 	R$ 7,00 	802g
-
Valor Total: R$ 28,00 
Carga Total: 1621g
Capacidade Ociosa: 44g

Carga da Mochila mochila_2 com capacidade de 629g
produto_7 	R$ 7,00 	543g
-
Valor Total: R$ 7,00 
Carga Total: 543g
Capacidade Ociosa: 86g
